# 📚 RAG Wissensdatenbank - Google Colab

Dieses Notebook installiert und startet die RAG Wissensdatenbank mit öffentlichem Zugang über Cloudflare Tunnel.

## Voraussetzungen
- **OpenAI API Key** als Colab Secret (Name: `OPENAI_API_KEY`)

### Secret einrichten
1. Links auf das 🔑 Symbol klicken
2. "Neues Secret hinzufügen"
3. Name: `OPENAI_API_KEY`, Wert: Ihr API Key
4. "Notebook-Zugriff" aktivieren

## 1️⃣ Repository klonen

In [ ]:
!git clone https://github.com/janschachtschabel/simple-document-rag.git
%cd simple-document-rag

## 2️⃣ Abhängigkeiten installieren

In [ ]:
!pip install -q -r requirements.txt
!pip install -q cloudflared

## 3️⃣ API Key laden und Modelle konfigurieren

In [ ]:
import os
from google.colab import userdata

# API Key aus Colab Secrets laden
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ OpenAI API Key geladen")
except Exception as e:
    print("❌ Secret 'OPENAI_API_KEY' nicht gefunden!")
    print("   🔑 Symbol links → Neues Secret → OPENAI_API_KEY")
    raise e

# Modelle (hier anpassen falls gewünscht)
OPENAI_MODEL = "gpt-4.1-mini"
EMBEDDING_MODEL = "text-embedding-ada-002"

os.environ["OPENAI_MODEL"] = OPENAI_MODEL
os.environ["EMBEDDING_MODEL"] = EMBEDDING_MODEL
os.environ["CHROMA_PERSIST_DIRECTORY"] = "./chroma_db"
os.environ["CHUNK_SIZE"] = "1000"
os.environ["CHUNK_OVERLAP"] = "200"
os.environ["TOP_K_RETRIEVAL"] = "5"

print(f"✅ LLM: {OPENAI_MODEL}")
print(f"✅ Embedding: {EMBEDDING_MODEL}")

## 4️⃣ FastAPI Server starten

In [ ]:
import subprocess
import time
import requests

api_process = subprocess.Popen(
    ["python", "main.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

print("⏳ Starte API...")
time.sleep(15)

try:
    r = requests.get("http://localhost:8000/health", timeout=5)
    print("✅ API läuft auf http://localhost:8000")
except:
    print("❌ API nicht erreichbar")

## 5️⃣ Streamlit + Cloudflare Tunnel starten

Nach Ausführung erscheint eine **öffentliche URL**.

In [ ]:
import subprocess
import threading
import re
import time

streamlit_process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
print("⏳ Starte Streamlit...")
time.sleep(5)

tunnel_process = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", "http://localhost:8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

def show_url():
    for line in tunnel_process.stderr:
        match = re.search(r'https://[\w-]+\.trycloudflare\.com', line)
        if match:
            print("\n" + "="*50)
            print("🎉 APP IST ONLINE!")
            print("="*50)
            print(f"🔗 {match.group(0)}")
            print("="*50)
            return

threading.Thread(target=show_url).start()
print("⏳ Warte auf Tunnel-URL...")

## 6️⃣ Status prüfen

In [ ]:
import requests

print("📊 Status")
print("-" * 30)

try:
    r = requests.get("http://localhost:8000/health", timeout=5)
    print(f"✅ API: OK ({r.json().get('statistics', {}).get('total_documents', 0)} Dokumente)")
except:
    print("❌ API: Offline")

try:
    r = requests.get("http://localhost:8501", timeout=5)
    print("✅ Streamlit: OK")
except:
    print("❌ Streamlit: Offline")

if 'tunnel_process' in dir() and tunnel_process.poll() is None:
    print("✅ Tunnel: Aktiv")
else:
    print("❌ Tunnel: Inaktiv")

## 🛑 Prozesse beenden

In [ ]:
try: api_process.terminate(); print("✅ API beendet")
except: pass
try: streamlit_process.terminate(); print("✅ Streamlit beendet")
except: pass
try: tunnel_process.terminate(); print("✅ Tunnel beendet")
except: pass

---
## 📝 Hinweise

- **Laufzeit**: Bis zu 12 Stunden (kostenlos)
- **Dokumente**: Gehen nach Sitzungsende verloren
- **Tunnel-URL**: Ändert sich bei jedem Neustart
- **Confluence**: In der App unter 🔷 Confluence konfigurieren